# 请求数据

In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [2]:
pattern = re.compile(r'\d+\. .*?')
url = "https://mp.weixin.qq.com/s?__biz=MzU4MTU1NzI4Mw==&mid=2247651261&idx=1&sn=ca4bfc8f0145198afeb6cc3d5ee08a66&chksm=fd4902f1ca3e8be77055a4158e9275f4802b27526cc9ef9e6728a2c0f262f242b5ea29886a40&scene=58&subscene=0#rd"
resp = requests.get(url)
soup = BeautifulSoup(resp.text, "html.parser")
body = soup.select("div#js_content > section[mpa-from-tpl=t]")[0]
paragraphs = [i.text.strip() for i in body if i.text.strip()]
result = []
content = []
for p in paragraphs:
    if pattern.match(p) and content:
        result.append({
            "title": content[0],
            "content": "\n".join(content[1:])
        })
        content = [p]
    else:
        content.append(p)
else:
    result.append({
        "title": content[0],
        "content": "\n".join(content[1:])
    })

result[-1]

{'title': '15. 与足浴协会共建商学院？湖南大学否认',
 'content': '6月12日，一段网络视频显示，湖南大学经济与贸易学院与湖南省足浴按摩行业协会共建商学院，揭牌仪式启动，受到网友关注。6月13日，湖南大学经济与贸易学院声明称，关注到网络上有关“湖南省足浴按摩行业协会与湖南大学经济贸易学院共建商学院揭牌仪式”的信息。经核查，学院未与该协会开展合作，该活动未经学院同意。湖南大学经济与贸易学院工作人员称，是有人冒用学校名义做的，目前正在调查取证中。目前正在取证中，会追究法律责任。\n湖南足浴按摩协会秘书长：对就业有好处，希望湖大提供智力支持\n据九派新闻，对于经济与贸易学院工作人员的否认，湖南省足浴按摩行业协会一工作人员称：“这肯定是真的，估计学院内部协调出现了问题，我也不太清楚对接人是谁。”另据新京报我们视频报道，对于此事，湖南足浴按摩行业秘书长刘先生接受采访时表示，（足浴是）劳动密集型的行业，对解决就业有很好的作用。希望湖大提供智力支持，规范协会发展。湖南有40多万足浴从业人员，希望社会对这个行业有更高更正确的认识。\nhttps://www.nbd.com.cn/articles/2023-06-13/2870348.html'}

# 计算 embedding 和 token_size

In [3]:
import openai
import tiktoken

In [4]:
model = "text-embedding-ada-002"
openai.api_key = "sk-xxxxx"
encoding = tiktoken.get_encoding("cl100k_base")

records = []
for i in result:
    resp = openai.Embedding.create(
        model=model,
        input=i["content"]
    )
    records.append({
        "title": i["title"],
        "content": i["content"],
        "embedding": resp["data"][0]["embedding"],
        "token_size": len(encoding.encode(i["content"]))
    })

len(records)

15

# 入库

In [6]:
import psycopg2
conn = psycopg2.connect(database="chi", host="localhost", user="chi", password="", port="5432")

cur = conn.cursor()
insert_query = """INSERT INTO docs (title, content, token_size, embedding) VALUES (%s, %s, %s, %s);"""

for i in records:
    cur.execute(insert_query, (i['title'], i['content'], i['token_size'], i['embedding']))

conn.commit()